In [207]:
import pickle as pkl
import nltk
import os
from datasets import load_dataset
from nltk.tokenize import sent_tokenize, word_tokenize
import multiprocessing
import evaluate
from rouge_score import rouge_scorer
from tqdm.auto import tqdm
from natsort import natsorted
import random
import pandas as pd
from datasets import load_dataset, DatasetDict, load_from_disk

In [2]:
pkls=os.listdir('cnn')
pkls=natsorted(pkls)
pkls_list=[]
for pikl in pkls:
    with open('cnn/{}'.format(pikl),'rb') as f:
        file=pkl.load(f)
    if not file:
        file.append(' ')
    pkls_list.extend(file)

print(len(pkls_list))
print(pkls_list[0])

11490
 Sure! Here is a summary of the article in three numbered sentences:

1. The Palestinian Authority officially became the 123rd member of the International Criminal Court (ICC) on Wednesday, giving the court jurisdiction over alleged crimes in Palestinian territories.
2. The ICC has initiated a preliminary examination into the situation in the Palestinian territories, potentially paving the way for investigations into possible war crimes committed by Israelis and Palestinians.
3. The accession to the ICC is seen as a move towards greater justice and accountability for alleged crimes committed in the occupied Palestinian territories, but both Israel and the United States have objected to the Palestinians' attempts to join the court.


In [3]:
count=0
for idx,item in enumerate(pkls_list):
    if item == " ":
        print(idx)
        count+=1
print(count)

13
29
56
98
187
197
216
236
237
242
259
297
304
306
347
364
397
404
408
409
430
437
456
492
493
574
582
678
716
725
727
770
805
817
827
841
842
861
865
902
904
993
1011
1018
1036
1037
1097
1221
1292
1349
1401
1505
1546
1566
1587
1673
1678
1680
2021
2024
2123
2227
2303
2325
2326
2373
2661
2663
2703
2764
2796
2958
2965
2975
2981
3111
3141
3247
3314
3329
3349
3417
3500
3574
3590
3779
4071
4168
4216
4342
4354
4464
4475
4483
4596
4632
4643
4658
4659
4662
4857
4865
4870
4884
4886
4941
4943
4949
4973
4978
4980
4987
4988
5010
5018
5043
5065
5070
5075
5109
5110
5147
5157
5178
5182
5185
5197
5198
5205
5211
5215
5228
5237
5240
5246
5247
5259
5270
5330
5343
5346
5373
5376
5395
5404
5407
5414
5427
5456
5469
5470
5471
5500
5507
5526
5579
5580
5602
5604
5605
5634
5641
5642
5668
5719
5730
5740
5765
5768
5770
5780
5793
5807
5814
5881
5893
5913
5919
5969
6000
6014
6054
6074
6083
6084
6104
6107
6115
6137
6181
6193
6212
6215
6225
6248
6255
6256
6264
6346
6347
6354
6361
6362
6378
6389
6395
6429
6458
6498
6

In [24]:
pkls_list=[orig_file[2]]

In [25]:
cnn_list=[]
for i in range(len(pkls_list)):
    temp=[]
    splits=pkls_list[i].split('\n')
    for split in splits:
        if split:
            if split[0].isdigit():
                try:
                    temp.append(split.split(".")[1].strip())
                except:
                    print(split)
    cnn_list.append(temp)

In [26]:
len(cnn_list)

1

In [27]:
cnn_list

[['Mohammad Javad Zarif is the Iranian Foreign Minister',
  "If you've been following the news lately, you know that he is U",
  'A recent search for "Mohammad Javad Zarif" and "jovial" yields thousands of results highlighting a misunderstanding of his personality, which tends to get overshadowed by the realities of his position, namely, his role in resolving the Alavi Foundation probe, and his role in the Iranian Revolution and subsequent time in the United States']]

In [6]:
cnn_list[200]

['Former New England Patriots player Aaron Hernandez is facing three more legal proceedings, including one criminal trial and two civil lawsuits, after being convicted of murder and other charges in the death of Odin Lloyd',
 'The double-murder trial for the deaths of Daniel de Abreu and Safiro Furtado is set to resume, with a new court date to be determined, and Hernandez is also facing civil suits from the families of the victims and a man who claims Hernandez shot him in a Miami strip club',
 "Hernandez's legal team has stated that he was unable to effectively defend himself during his trial in Massachusetts due to various factors, and the NFL players union has filed a grievance over unpaid bonus money on behalf of Hernandez"]

In [34]:
# with open('reddit.pkl', 'wb') as f:
#     pkl.dump(cnn_list,f) 

In [56]:
dataset = load_dataset("cnn_dailymail", '3.0.0')
article_key = 'article'
summary_key = 'highlights'
dataset=dataset['test']

In [73]:
dataset = load_dataset("EdinburghNLP/xsum")
article_key = 'document'
summary_key = 'summary'
dataset=dataset['test']

In [264]:
dataset = load_dataset("argilla/news-summary")
article_key = 'text'
summary_key = 'prediction'
dataset = DatasetDict({
    'train': dataset['test'],
    'test': dataset['train']})

dataset=dataset['test']

In [241]:
data = load_dataset('reddit_tifu', 'long')
article_key = 'documents'
summary_key = 'tldr'
    # 80% train, 20% test + validation
train_testvalid = data['train'].train_test_split(test_size=0.2, seed=42)
# Split the 20% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42)
# gather everyone if you want to have a single DatasetDict
dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

dataset=dataset['test']

In [4]:
articles=dataset.select(range(10))[article_key]

In [294]:
articles=dataset.select([0,1,2,3,7,8,10,11,12,16])[article_key]

In [266]:
articles

['PHNOM PENH (Reuters) - Sweden said on Tuesday it was stopping new aid for Cambodia, except in education and research, and would no longer support a reform programme after the main opposition party was outlawed by the Supreme Court at the government s request. The announcement marked the first concrete action by a European Union country in protest at a political crackdown in which veteran Prime Minister Hun Sen s main rival has also been arrested and civil rights groups and independent media attacked. The United States cut election funding and said it would take more punitive steps after last week s ban on the Cambodia National Rescue Party (CNRP). The European Union has also threatened action. Sweden s embassy in Phnom Penh said the country was reviewing its engagement with Cambodia.   We will not initiate any new government-to-government development cooperation agreements, except in the areas of education and research,  it said in a statement. As a consequence, it would be unable to

In [295]:
for k,v in enumerate(articles):
    articles[k]=v.replace('\n',"")

In [268]:
gold=dataset.select(range(10))[summary_key]

In [296]:
gold=dataset.select([0,1,2,3,7,8,10,11,12,16])[summary_key]

In [164]:
articles

['well this was around 9 years ago when i was a 2nd grader. so basically it was some kids birthday and since it was primary school most kids brought cupcakes and the teacher would pass it out to the class. me being the hungry little shit that i was i was desperately in the mood for some vanilla cupcakes but the teacher decided to pass out the chocolate ones first. after she had finished passing out the chocolate ones she picks up the vanilla ones and says, "raise your hand if you want vanilla!". at that moment i was talking to a friend and it took me a bit to process the fact that she just asked who wanted vanilla. i proceed to raise my hand and it turns out the cupcakes were right above me. i knocked the cupcakes out of her hand and they spilled  all over her. she got bat-shit angry and begins to make a scene in the middle of the whole cafeteria. she explained how i "ruined her brand new shoes" and the bitch even asked me if i had an allowance so i could pay for her shoes. she had ask

In [165]:
gold

['** i wanted vanilla cupcakes so i raised my hand and knocked a whole box of cupcakes all over my teacher.',
 'i shit my pants after eating a lot of grease.',
 'asked my girlfriend to wake up with a blowjob and instead of waking up to a glorious orgasm, i filled her mouth with enough piss to make bear grylls flinch.',
 "i nearly crashed into some parked cars and a fire truck in chelsea in a bizarre situation that wasn't included on my driving test, probably due to it being incredibly specific.",
 'ball stuck in body from butt play masturbation.',
 'i hate people so i wandered off by myself in the largest city in costa rica, got very lost, got help, then got very lost again. also, my phone was dead.**',
 'bought french fries, attacked by  seagulls, got scared and started swearing next to a park, everyone was laughing at me, threw my fries and ran.',
 'tried to make a quesadilla, ended up burning my dog and disgracing my family in front of out fundamentalist neighbors.',
 "cut/scraped m

In [297]:
gold_new=[]
for i in range(10):
    print(gold[i][0]['text'])
    gold_new.append(gold[i][0]['text'])
    

Sweden stops some new aid for Cambodia in protest over crackdown
U.N. chief calls for united, 'appropriate action' on North Korea
House tax positions begin to emerge ahead of talks with Senate
Russia suspends diplomatic presence in Yemen, moves envoy to Riyadh: agencies
Ex-Michigan lawmaker to stand trial; charges dismissed against another
JAL to screen majority-muslim country passengers before departure for U.S.
Trump wins Christie backing, marches toward Super Tuesday
Iceland sets snap election for Oct. 28: president
Ex-Georgian leader risks extradition on return to Ukraine
Trump extends program allowing some veterans to use local doctors, hospitals


In [271]:
gold_new


['Sweden stops some new aid for Cambodia in protest over crackdown',
 'Russia suspends diplomatic presence in Yemen, moves envoy to Riyadh: agencies',
 'Top Senate Republican tells Supreme Court nominee: Senate will not act',
 'JAL to screen majority-muslim country passengers before departure for U.S.',
 'Iceland sets snap election for Oct. 28: president',
 'Women in male-dominated career fields watch a unique U.S. presidential campaign',
 'Armed group seeks legitimacy with Tripoli migrant deal, source says',
 'Trump extends program allowing some veterans to use local doctors, hospitals',
 'London metro station incident caused by bomb, top UK police officer says',
 "Russia, Saudi Arabia cement new friendship with king's visit"]

In [244]:
for k,v in enumerate(gold):
    gold[k]=v.replace('\n',"")

In [298]:
for k,v in enumerate(gold_new):
    gold_new[k]=v.replace('\n',"")

In [299]:
with open('news_capped_random.pkl', 'rb') as f:
    para_file=pkl.load(f) 
    

In [300]:
paraphrased=[]
#for i in range(10):
for i in [0,1,2,3,7,8,10,11,12,16]:
    paraphrased.append(" ".join(para_file[i]))
    

In [284]:
para_file[14]

['']

In [301]:
paraphrased

[' The CNRP was banned after its leader',
 ' Such a response must include wise and bold diplomacy to be effective,  Feltman told the council',
 ' "House members',
 'Due to the unstable situation in the capital city of Sanaa, Russia has temporarily withdrawn its diplomatic personnel from Yemen',
 'A judge in Michigan dismissed charges against a former state representative, Todd Courser, on Tuesday, while ruling that another lawmaker should stand trial for attempting to cover up an extramarital affair',
 "Japan Airlines has started screening passengers from seven Muslim-majority countries before they depart for the United States, as required by President Donald Trump's travel ban",
 ' presidential election - although Clinton has yet to secure her party’s nomination',
 "Iceland will hold an unexpected parliamentary election on October 28th following the collapse of the current government due to a scandal involving the prime minister's father",
 ' Saakashvili has been involved in a campaig

In [302]:
with open('../data_original/reddit_capped_random.pkl', 'rb') as f:
    orig_file=pkl.load(f) 

In [303]:
orig_file_new=[]
for i in range(len(orig_file)):
    try:
        temp=[]
        splits=orig_file[i].split('\n')
        #print(splits)
        if len(splits)>1:
            for split in splits:
                if split:
                    if split[0].isdigit():
                        try:
                            temp.append(split.split(".")[1].strip())
                        except:
                            #print(split)
                            pass
            orig_file_new.append(" ".join(temp))
        else:
            orig_file_new.append(orig_file[i])
            
    except:
        orig_file_new.append(orig_file[i])

In [304]:
original=[]
#for i in range(10):
for i in [0,1,2,3,7,8,10,11,12,16]:
    original.append("".join(orig_file_new[i]))

In [305]:
gold[4]

[{'score': 1.0,
  'text': 'Ex-Michigan lawmaker to stand trial; charges dismissed against another'}]

In [306]:
dic={'Article':articles, 'Paraphrased':paraphrased, 'Original': original, 'Gold':gold_new }

In [262]:
df=pd.DataFrame.from_dict(dic)

In [307]:
df.to_csv('../annotation/dolly-news.csv', index=False)

In [218]:
paraphrased

['well this was around 9 years ago when i was a 2nd grader. so basically it was some kids birthday and since it was primary school most kids brought cupcakes and the teacher would pass it out to the class. me being the hungry little shit that i was i was desperately in the mood for some vanilla cupcakes but the teacher decided to pass out the chocolate ones first. after she had finished passing out the chocolate ones she picks up the vanilla ones and says, "raise your hand if you want vanilla!". at that moment i was talking to a friend and it took me a bit to process the fact that she just asked who wanted vanilla. I reached up and found the cupcakes were conveniently located just above me. I accidentally knocked the cupcakes out of her hand, causing them to spill all over her. she got bat-shit angry and begins to make a scene in the middle of the whole cafeteria. she explained how i "ruined her brand new shoes" and the bitch even asked me if i had an allowance so i could pay for her s

In [124]:
gold

['There is a "chronic" need for more housing for prison leavers in Wales, according to a charity.',
 'A man has appeared in court after firearms, ammunition and cash were seized by police in Edinburgh.',
 'Four people accused of kidnapping and torturing a mentally disabled man in a "racially motivated" attack streamed on Facebook have been denied bail.',
 'West Brom have appointed Nicky Hammond as technical director, ending his 20-year association with Reading.',
 'The pancreas can be triggered to regenerate itself through a type of fasting diet, say US researchers.',
 'A "medal at any cost" approach created a "culture of fear" at British Cycling, says former rider Wendy Houvenaghel.',
 'Have you heard the one about the computer programmer who bought a failing comedy club in Texas and turned it into a million dollar a year business?',
 "The reaction from BT's investors told us much about media regulator Ofcom's ruling on the fate of Openreach, the BT subsidiary that provides much of th